In [ ]:
import numpy as np
import pandas as pd
from math import * 
from pathlib import Path
from matplotlib import pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [ ]:
def gnss_log_to_dataframes(path):
    print('Loading ' + path, flush=True)
    gnss_section_names = {'Raw','UncalAccel', 'UncalGyro', 'UncalMag', 'Fix', 'Status', 'OrientationDeg'}
    with open(path) as f_open:
        datalines = f_open.readlines()

    datas = {k: [] for k in gnss_section_names}
    gnss_map = {k: [] for k in gnss_section_names}
    for dataline in datalines:
        is_header = dataline.startswith('#')
        dataline = dataline.strip('#').strip().split(',')
        # skip over notes, version numbers, etc
        if is_header and dataline[0] in gnss_section_names:
            try:
                gnss_map[dataline[0]] = dataline[1:]
            except:
                pass
        elif not is_header:
            try:
                datas[dataline[0]].append(dataline[1:])
            except:
                pass
    results = dict()
    for k, v in datas.items():
        results[k] = pd.DataFrame(v, columns=gnss_map[k])
    # pandas doesn't properly infer types from these lists by default
    for k, df in results.items():
        for col in df.columns:
            if col == 'CodeType':
                continue
            try:
                results[k][col] = pd.to_numeric(results[k][col])
            except:
                pass
    return results

In [ ]:
# lowpass filter

from scipy.signal import butter, lfilter

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

order = 3
fs = 50.0
cutoff = 2.5

# Overview


This notebook calculate the direction of driving with geomagnetic and accelerometers. and compare the value obtained by this with the direction calculated from the transition of latitude and longitude of the ground truth.

In [ ]:
# Offset correction
# refarence https://github.com/J-ROCKET-BOY/SS-Fitting

def SS_fit(data) : 

    x = data[:,[0]]
    y = data[:,[1]]
    z = data[:,[2]]

    data_len = len(x)
    
    x2 = np.power(x,2)
    y2 = np.power(y,2)
    z2 = np.power(z,2)

    r1 = -x*(x2+y2+z2)
    r2= -y*(x2+y2+z2)
    r3 = -z*(x2+y2+z2)
    r4 = -(x2+y2+z2)

    left = np.array([[np.sum(x2),np.sum(x*y),np.sum(x*z),np.sum(x)],
                     [np.sum(x*y),np.sum(y2),np.sum(y*z),np.sum(y)],
                     [np.sum(x*z),np.sum(y*z),np.sum(z2),np.sum(z)],
                     [np.sum(x), np.sum(y), np.sum(z), data_len]])
    
    right = np.array([np.sum(r1),
                      np.sum(r2),
                      np.sum(r3),
                      np.sum(r4)])
    
    si = np.dot(np.linalg.inv(left),right)

    x0 = (-1/2)* si[0]
    y0 = (-1/2)* si[1]
    z0 = (-1/2)* si[2]
    
    return np.array([x0,y0,z0])

In [ ]:
# Vincenty's formulae
# refarence https://qiita.com/r-fuji/items/99ca549b963cedc106ab

def vincenty_inverse(lat1, lon1, lat2, lon2):

    # Not advanced
    if isclose(lat1, lat2) and isclose(lon1, lon2):
        return False
    
    # WGS84
    a = 6378137.0
    ƒ = 1 / 298.257223563
    b = (1 - ƒ) * a

    lat_1 = atan((1 - ƒ) * tan(radians(lat1)))
    lat_2 = atan((1 - ƒ) * tan(radians(lat2)))
    
    lon_diff = radians(lon2) - radians(lon1)
    λ = lon_diff

    for i in range(1000):
        sinλ = sin(λ)
        cosλ = cos(λ)
        sinσ = sqrt((cos(lat_2) * sinλ) ** 2 + (cos(lat_1) * sin(lat_2) - sin(lat_1) * cos(lat_2) * cosλ) ** 2)
        cosσ = sin(lat_1) * sin(lat_2) + cos(lat_1) * cos(lat_2) * cosλ
        σ = atan2(sinσ, cosσ)
        sinα = cos(lat_1) * cos(lat_2) * sinλ / sinσ
        cos2α = 1 - sinα ** 2
        cos2σm = cosσ - 2 * sin(lat_1) * sin(lat_2) / cos2α
        C = ƒ / 16 * cos2α * (4 + ƒ * (4 - 3 * cos2α))
        λʹ = λ
        λ = lon_diff + (1 - C) * ƒ * sinα * (σ + C * sinσ * (cos2σm + C * cosσ * (-1 + 2 * cos2σm ** 2)))
        
        if abs(λ - λʹ) <= 1e-12:
            break
    else:
        return None

    α = atan2(cos(lat_2) * sinλ, cos(lat_1) * sin(lat_2) - sin(lat_1) * cos(lat_2) * cosλ)

    if α < 0:
        α = α + pi * 2

    return degrees(α)

In [ ]:
def calc3(row):
    deg = - degrees(atan2(-1*row['calc2'],row['calc1']))
    if deg < 0:
        deg += 360
    return deg 

In [ ]:
data_dir = Path('../input/google-smartphone-decimeter-challenge')
logg_files = (data_dir / 'train').rglob('*Pixel4_GnssLog.txt')
logg_paths = [str(p) for p in logg_files]

In [ ]:
# use acce and magn

fig,ax = plt.subplots(len(logg_paths)*2, 1, figsize=(25,6*2*len(logg_paths)))

for n, logg_path in enumerate(logg_paths):
                      
#     if n > 3:
#         break
    path = logg_path
    gt_path = path.split("Pixel4_")[0]+"ground_truth.csv"

    gt_df = pd.read_csv(gt_path)
    sample = gnss_log_to_dataframes(str(path))

    acce_df = sample["UncalAccel"]
    mag_df = sample["UncalMag"]

    if not len(acce_df):
        continue
    
    acce_df["millisSinceGpsEpoch"] = acce_df["utcTimeMillis"] - 315964800000
    mag_df["millisSinceGpsEpoch"] = mag_df["utcTimeMillis"] - 315964800000
    
#     acce filtering and smooting
    
    acce_df["global_x"] = acce_df["UncalAccelZMps2"]
    acce_df["global_y"] = acce_df["UncalAccelXMps2"]
    acce_df["global_z"] = acce_df["UncalAccelYMps2"]
    
    acce_df["x_f"] = butter_lowpass_filter(acce_df["global_x"], cutoff, fs, order)
    acce_df["y_f"] = butter_lowpass_filter(acce_df["global_y"], cutoff, fs, order)
    acce_df["z_f"] = butter_lowpass_filter(acce_df["global_z"], cutoff, fs, order)
    
    smooth_range = 1000

    acce_df["x_f"] = acce_df["x_f"].rolling(smooth_range, center=True, min_periods=1).mean()
    acce_df["y_f"] = acce_df["y_f"].rolling(smooth_range, center=True, min_periods=1).mean()
    acce_df["z_f"] = acce_df["z_f"].rolling(smooth_range, center=True, min_periods=1).mean()
    
#     magn filtering and smooting , offset correction
    
    mag_df["global_mx"] = mag_df["UncalMagZMicroT"]
    mag_df["global_my"] = mag_df["UncalMagYMicroT"]
    mag_df["global_mz"] = mag_df["UncalMagXMicroT"]
    
    smooth_range = 1000
    
    mag_df["global_mx"] = mag_df["global_mx"].rolling(smooth_range,  min_periods=1).mean()
    mag_df["global_my"] = mag_df["global_mz"].rolling(smooth_range,  min_periods=1).mean()
    mag_df["global_mz"] = mag_df["global_my"].rolling(smooth_range,  min_periods=1).mean()
    
    offset = SS_fit(np.array(mag_df[["global_mx","global_my","global_mz"]]))
    mag_df["global_mx"] = (mag_df["global_mx"] - offset[0])*-1
    mag_df["global_my"] = mag_df["global_my"] - offset[1]
    mag_df["global_mz"] = mag_df["global_mz"] - offset[2]

#     merge the value of the one with the closest time 
    
    acce_df["millisSinceGpsEpoch"] = acce_df["millisSinceGpsEpoch"]//1000 +10
    mag_df["millisSinceGpsEpoch"] = mag_df["millisSinceGpsEpoch"]//1000 +10
    gt_df["millisSinceGpsEpoch"] = gt_df["millisSinceGpsEpoch"]//1000
    
    acce_df = pd.merge_asof(acce_df.sort_values('millisSinceGpsEpoch'), 
                           mag_df[["global_mx", "global_my","global_mz","millisSinceGpsEpoch"]].sort_values('millisSinceGpsEpoch') ,on='millisSinceGpsEpoch', direction='nearest')
    
    acce_df = pd.merge_asof(gt_df[["millisSinceGpsEpoch","latDeg","lngDeg"]].sort_values('millisSinceGpsEpoch'), 
                           acce_df[["millisSinceGpsEpoch", "x_f","y_f","z_f","global_mx","global_my","global_mz"]].sort_values('millisSinceGpsEpoch') ,on='millisSinceGpsEpoch', direction='nearest')

#     as a sensor value when stopped
    
    start_mean_range = 10
    
    x_start_mean = acce_df[:start_mean_range]["x_f"].mean()
    y_start_mean = acce_df[:start_mean_range]["y_f"].mean()
    z_start_mean = acce_df[:start_mean_range]["z_f"].mean() 
    
#     roll and picth, device tilt
    
    r = atan(y_start_mean/z_start_mean)
    p = atan(x_start_mean/(y_start_mean**2 + z_start_mean**2)**0.5)
    
#     calculation　degrees
    
    acce_df["calc1"] = acce_df["global_mx"]*cos(p) + acce_df["global_my"]*sin(r)*sin(p) + acce_df["global_mz"]*sin(p)*cos(r)
    acce_df["calc2"] = acce_df["global_mz"]*sin(r) - acce_df["global_my"]*cos(r)
    acce_df["calc_deg"] = acce_df.apply(calc3, axis=1)
    
#     degrees with lat and lng by ground truth

    len_df = len(acce_df)
    acce_df["deg"] = 0
    
    gt_lat_prev = 0
    gt_lng_prev = 0
    
    for i in range(1,len_df):
        if i > 1:
            res = vincenty_inverse(gt_lat_prev,gt_lng_prev,acce_df["latDeg"].loc[i],acce_df["lngDeg"].loc[i])
            if res:
                acce_df["deg"].loc[i] = res
            else:
                if i > 0:
                    acce_df["deg"].loc[i] = acce_df["deg"].loc[i-1]
                else:
                    acce_df["deg"].loc[i] = 0
        
        gt_lat_prev = acce_df["latDeg"].loc[i]
        gt_lng_prev = acce_df["lngDeg"].loc[i]
    
    ax[2*n].plot(acce_df["millisSinceGpsEpoch"],acce_df["global_mx"] ,label="mx")
    ax[2*n].plot(acce_df["millisSinceGpsEpoch"],acce_df["global_my"], label="my")
    ax[2*n].plot(acce_df["millisSinceGpsEpoch"],acce_df["global_mz"], label="mz")
    ax[2*n].legend()
    ax[2*n].set_title(logg_path)
 
    ax[2*n+1].plot(acce_df["millisSinceGpsEpoch"],acce_df["deg"] ,label="deg_by_gt")
    ax[2*n+1].plot(acce_df["millisSinceGpsEpoch"],acce_df["calc_deg"] ,label="deg_by_mag")    
    ax[2*n+1].legend()
    ax[2*n+1].set_title("deg")
    